# Utils

> Fill in a module description here


In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
from fastcore.utils import *  # type: ignore # noqa: F403
from torch.utils.data import DataLoader # type: ignore # noqa: F403
import yaml
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import torch
from fedai.data import * # noqa: F403
from fedai.vision.models import * # noqa: F403


In [ ]:
#| export
import importlib
def get_class(module_name, class_name):
    module = importlib.import_module(module_name)
    return getattr(module, class_name)

In [ ]:
#| export
def get_server(cfg, lst_data_dict, model, holdout_ds, **kwargs):
    Server = get_class('fedai.servers', f'Server_{cfg.name}')
    client_class = get_class('fedai.clients', f'Client_{cfg.name}')
    return Server(cfg, lst_data_dict, model, holdout_ds, client_class, **kwargs)

In [ ]:
#| export
def load_config(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

In [ ]:
#| export
def save_space(client) -> None:
    client.clear_model()
    del client.optimizer
    del client
    import gc
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

Because of the difference that it takes to prepare the dataset for sinle device vs multi-device training, we make a method that handles this separately. `prepare_dl` prepares the dataloader needed for the trainer's type.

In [ ]:
#| export
def prepare_dl(cfg, ds, shuffle=True, collate_fn=None):
    return DataLoader(
        ds,
        batch_size= cfg.data.batch_size,
        shuffle= shuffle,
        collate_fn= collate_fn     
    )

## Graph utils

In [ ]:
#| export
def generate_graph(K, # (np.ndarray): The input matrix.
                   symmetrize=True, # (bool): If True, makes the matrix symmetric.
                   normalize=True, # (bool): If True, normalizes the matrix symmetrically.
                   threshold= 0, # (float or None): If provided, sets values below this threshold to 0.
                   diag_fill= 0): # (float or None): If provided, fills the diagonal with this value.
    
    graph = np.random.randn(K, K)

    # Symmetrize the matrix
    if symmetrize:
        graph = (graph + graph.T) / 2

    # Apply threshold
    if threshold is not None:
        graph = np.where(graph > threshold, graph, 0)

    # Normalize the matrix symmetrically
    if normalize:
        row_sums = graph.sum(axis=1, keepdims=True)
        col_sums = graph.sum(axis=0, keepdims=True)
        norm_factor = np.sqrt(row_sums @ col_sums)  # Symmetric normalization factor
        graph = np.divide(graph, norm_factor, where=norm_factor != 0)

    # Fill the diagonal
    if diag_fill is not None:
        np.fill_diagonal(graph, diag_fill)

    return graph


In [ ]:
#| export
def draw_nx_graph(graph):

    # Create a NetworkX graph
    G = nx.Graph()

    # Add weighted edges
    for i in range(graph.shape[0]):
        for j in range(graph.shape[1]):
            if graph[i, j] > 0:  # Add edge only if weight > 0
                G.add_edge(i, j, weight=graph[i, j])

    # Get edge weights for coloring
    edges = G.edges(data=True)
    weights = [d['weight'] for _, _, d in edges]

    # Normalize weights for coloring (between 0 and 1)
    normalized_weights = (weights - np.min(weights)) / (np.max(weights) - np.min(weights) + 1e-8)

    # Draw the graph
    pos = nx.spring_layout(G)  # Spring layout for positioning
    plt.figure(figsize=(8, 8))

    # Draw nodes
    nx.draw_networkx_nodes(G, pos, node_size=500, node_color="indigo")

    # Draw edges with color mapping
    edges = nx.draw_networkx_edges(
        G, pos, edge_color=normalized_weights, edge_cmap=plt.cm.viridis, width=2
    )

    # Draw labels
    nx.draw_networkx_labels(G, pos, font_size=12, font_color="black")

    # Add a colorbar
    sm = plt.cm.ScalarMappable(cmap=plt.cm.viridis, norm=plt.Normalize(vmin=np.min(weights), vmax=np.max(weights)))
    sm.set_array([])  # This line fixes the issue
    plt.colorbar(sm, label="Edge Weight", ax=plt.gca())

    plt.title("Graph Connections", fontsize=12)
    plt.axis("off")
    plt.show()

In [ ]:
#| export
def draw_matrix(graph):
    # draw adjacency matrix represntation of a graph
    K = graph.shape[0]

    plt.figure(figsize=(8, 8))
    plt.imshow(graph, cmap='viridis', interpolation='nearest')

    # Add ticks for all nodes
    plt.xticks(ticks=np.arange(K), labels=np.arange(K))
    plt.yticks(ticks=np.arange(K), labels=np.arange(K))

    # Add labels, colorbar, and title
    plt.colorbar(label="Connection Weight")
    plt.title("", fontsize=16)
    plt.xlabel("Node Index")
    plt.ylabel("Node Index")

    # Annotate the weights in the matrix
    for i in range(graph.shape[0]):
        for j in range(graph.shape[1]):
            if graph[i, j] > 0:  # Show only non-zero weights
                plt.text(j, i, f"{graph[i, j]:.3f}", ha='center', va='center', color='white')

    plt.show()


In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()